In [1]:
import pandas as pd
import numpy as np

# ----------------------------
# 1) Lecture du CSV annuel
# ----------------------------
CSV_PATH = "../data/logistics/logistics-data-interpolated.csv"
df_annual = pd.read_csv(CSV_PATH)

# ----------------------------
# 2) Paramètres mensuels (Saisonnalités)
# ----------------------------

# MODE NORMAL : Pic hivernal classique, creux en août
MONTH_PCT_NORMAL = {
    1: 10, 2: 9, 3: 9, 4: 8, 5: 7, 6: 8, 
    7: 7, 8: 6, 9: 9, 10: 10, 11: 9, 12: 8
}

# MODE CRISE : Explosion en Mars/Avril (Vague épidémique)
# On simule ici que 40% de l'activité annuelle se concentre sur le printemps
MONTH_PCT_CRISIS = {
    1: 5, 2: 5, 3: 20, 4: 25, 5: 10, 6: 5, 
    7: 5, 8: 5, 9: 5, 10: 5, 11: 5, 12: 5
}

def validate_pct(pct_dict, name):
    if sum(pct_dict.values()) != 100:
        raise ValueError(f"Total {name} doit faire 100. Actuel={sum(pct_dict.values())}")

validate_pct(MONTH_PCT_NORMAL, "NORMAL")
validate_pct(MONTH_PCT_CRISIS, "CRISIS")

# ----------------------------
# 2b) Arrondi par unité
# ----------------------------
ROUNDING_BY_UNIT = {
    "colis": 0, "repas": 0, "kg": 0, "t": 2, "km": 1, "m2": 1
}
DEFAULT_DECIMALS = 2

# ----------------------------
# 3) Fonction de répartition améliorée
# ----------------------------
def annual_to_daily(
    annual_value: float,
    year: int,
    month_pct: dict,
    weekend_factor: float = 0.80,
    weekday_factor: float = 1.05,
    noise_sigma: float = 0.08,
    seed: int = 42
) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    dates = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="D")
    df = pd.DataFrame({"date": dates})
    df["month"] = df["date"].dt.month
    df["dow"] = df["date"].dt.weekday

    values = np.zeros(len(df), dtype=float)

    for m in range(1, 13):
        idx = df.index[df["month"] == m].to_numpy()
        monthly_total = float(annual_value) * (month_pct[m] / 100.0)

        # Application du poids semaine/week-end
        base = np.where(df.loc[idx, "dow"].to_numpy() >= 5, weekend_factor, weekday_factor).astype(float)
        
        # Ajout du bruit aléatoire
        noise = np.exp(rng.normal(loc=0.0, scale=noise_sigma, size=len(idx)))
        weights = base * noise
        
        values[idx] = monthly_total * (weights / weights.sum())

    df["value"] = values
    return df

# ----------------------------
# 4) Génération des données
# ----------------------------
COEF_CRISE_VOLUME = 1.70 # Augmentation du volume global en plus de la saisonnalité
all_daily = []

for i, row in df_annual.iterrows():
    year = int(row["ANNEE"])
    indic = row.get("INDICATEUR", "")
    unite = row.get("UNITE", "")

    for site in ["PLF", "CFX"]:
        annual_val = row[site]
        if pd.isna(annual_val): continue

        # --- GENERATION MODE NORMAL ---
        df_normal = annual_to_daily(annual_val, year, MONTH_PCT_NORMAL)
        
        # --- GENERATION MODE CRISE ---
        # On applique la tendance mensuelle de crise ET le coefficient de volume
        df_crisis = annual_to_daily(annual_val * COEF_CRISE_VOLUME, year, MONTH_PCT_CRISIS)

        # Fusion des résultats
        df_final = df_normal.copy()
        df_final["value_crise"] = df_crisis["value"]
        
        # Métadonnées
        df_final["site_code"] = site
        df_final["indicateur"] = indic
        df_final["unite"] = unite
        
        # Arrondis
        dec = ROUNDING_BY_UNIT.get(str(unite).strip(), DEFAULT_DECIMALS)
        df_final["value"] = df_final["value"].round(dec)
        df_final["value_crise"] = df_final["value_crise"].round(dec)

        all_daily.append(df_final)

df_daily_all = pd.concat(all_daily, ignore_index=True)

# ----------------------------
# 5) Export & Check
# ----------------------------
OUT_PATH = "../data/logistics/logistics-daily-crisis-comparison.csv"
df_daily_all.to_csv(OUT_PATH, index=False)

print(f"✅ Simulation terminée.")
print(f"Moyenne journalière Normale: {df_daily_all['value'].mean():.2f}")
print(f"Moyenne journalière Crise: {df_daily_all['value_crise'].mean():.2f}")

✅ Simulation terminée.
Moyenne journalière Normale: 605.60
Moyenne journalière Crise: 1029.52


In [2]:
df_daily_all["indicateur"].value_counts()
df_daily_all.groupby(["site_code", "indicateur"]).size()

site_code  indicateur  
CFX        Déchets         14248
           Hygiène          8768
           Lingerie         4384
           Magasin          6576
           Restauration     4384
           Vaguemestre      8768
PLF        Déchets         14248
           Hygiène          8768
           Lingerie         4384
           Magasin          6576
           Restauration     4384
           Vaguemestre      8768
dtype: int64

In [3]:
# ============================
# Graph: Déchets / Cartons (journalier)
# - Charge le CSV journalier reconstitué
# - Filtre Déchets + Cartons
# - Choisit un site (PLF/CFX) et une année
# - Trace la courbe + moyenne mobile 7 jours
# ============================

import pandas as pd
import matplotlib.pyplot as plt

# 1) Chemin vers TON fichier journalier généré
DAILY_CSV_PATH =  "../data/patients/patients_donnees_journalieres_reconstituees.csv"  # adapte si besoin

df = pd.read_csv(DAILY_CSV_PATH)

# 2) Parse de la date
df["date"] = pd.to_datetime(df["date"])

# 3) Paramètres du graphe
SITE = "PLF"         # "PLF" ou "CFX"
YEAR = 2012          # année à visualiser

# 4) Filtre: 
mask = (
    (df["site_code"] == SITE) &
    (df["year"] == YEAR) &
    (df["indicateur"].str.strip() == "Urgences") &
    (df["sous_indicateur"].str.strip().str.lower() == "Passages")
)

cartons = df.loc[mask, ["date", "value", "unite"]].sort_values("date").copy()

if cartons.empty:
    print("❌ Aucune donnée trouvée pour Déchets / Cartons avec ces filtres.")
    print("➡️ Vérifie les valeurs exactes de 'indicateur' et 'sous_indicateur' avec :")
    print('   df[df["indicateur"]=="Déchets"]["sous_indicateur"].unique()')
else:
    unit = cartons["unite"].iloc[0] if "unite" in cartons.columns and len(cartons) > 0 else ""
    cartons["mm7"] = cartons["value"].rolling(7, min_periods=1).mean()

    # 5) Plot
    plt.figure(figsize=(14, 5))
    plt.plot(cartons["date"], cartons["value"], label="Journalier")
    plt.plot(cartons["date"], cartons["mm7"], label="Moyenne mobile 7j")
    plt.title(f"Déchets - Cartons ({SITE}) - {YEAR}")
    plt.xlabel("Date")
    plt.ylabel(f"Volume ({unit})" if unit else "Volume")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # (Optionnel) Aperçu
    display(cartons.head(10))

❌ Aucune donnée trouvée pour Déchets / Cartons avec ces filtres.
➡️ Vérifie les valeurs exactes de 'indicateur' et 'sous_indicateur' avec :
   df[df["indicateur"]=="Déchets"]["sous_indicateur"].unique()
